In [1]:
import os
os.chdir('/home')

from lib.labels import LabelHandler
from lib.util import dict_sample

FINDING: 0.4% of labels have no associated text

maybe we can infer these later by using the graph and the vector space?

In [4]:
label_handler = LabelHandler('train', 30)
text_labels = label_handler.get()
#glove_labels = label_handler.get_glove()
label_graph = label_handler.get_graph(identity=False)

Starting _load_labels...
_load_labels took 0.0002739429473876953 seconds


In [5]:
label_graph

array([[0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])